In [1]:
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, ZoomControl, Marker, Popup
from ipywidgets import Output, HBox,VBox, HTML
import bqplot.pyplot as plt
from bqplot import DateScale, LinearScale,OrdinalScale, Bars, Axis, Figure,CATEGORY10
from geopy.geocoders import Nominatim
import asyncio
from ipyflex import FlexLayout 
import json 
import os
import configparser
import json
import random
import requests
import Functions as fct
config = configparser.ConfigParser() 
config.read('.config')
token = config['DEFAULT']['token']


In [2]:
#Add countries.geojson layer
if not os.path.exists('countries.geojson'):
    url = 'https://github.com/datasets/geo-countries/blob/master/data/countries.geojson'
    r = requests.get(url)
    with open('countries_copy', 'w') as f:
        f.write(r.content.decode("utf-8"))
   
with open('countries.geojson', 'r') as f:
    data = json.load(f)
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }
geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)
m = Map(center=(50.6252978589571, 0.34580993652344), zoom=2)

geo_json_layer = GeoJSON(data=geo_json.data)
m.add_layer(geo_json_layer)


In [8]:
'''import ipywidgets as widgets
from ipyleaflet import Map, GeoJSON, Popup


# Create a Popup
popup = Popup(
    location=[],
    child=widgets.HTML(""),
    close_button=False,
    auto_close=False,
    close_on_escape_key=False,
)
# Keep track of the currently displayed Popup
current_popup = None

# Function to handle hover event
def handle_hover(event, feature, **kwargs):
    global current_popup
    country = feature['properties']['ADMIN']
    coordinates = feature['geometry']['coordinates']
    coord = coordinates[0][0][0]
    latitude, longitude = coord[1], coord[0]
    popup.location = [latitude, longitude]
    popup.child.value = country 

    # Remove it existing Popup
    if current_popup is not None:
        m.remove_layer(current_popup)

    # Add the updated Popup to the map
    m.add_layer(popup)

    # Update the currently displayed Popup
    current_popup = popup

geo_json_layer.on_hover(handle_hover)'''



In [ ]:
box = VBox()
axes_options_1 = {
    "x": {"label": "Time","label_style": {"font-size": "3px"}},
    "y": {"label": "Power Consumption (MWh)", "label_offset": "6ex","label_font_size": "3px"},
}
axes_options_2 = {
    "x": {"label": "Time","label_offset": "6ex"},
    "y": {"label": "Power Value", "label_offset": "6ex"},
}
axes_options_3 = {
    "x": {"label": "Source Energie","label_offset": "6ex"},
    "y": {"label": "Value (MWh)", "label_offset": "6ex"},
}
axes_options_4 = {
    "x": {"label": "Country","label_offset": "6ex"},
    "y": {"label": "Value (MWh)", "label_offset": "6ex"},
}

In [5]:
#last merge zith 50%



############################# Hourly_Power_Consumption_fig#########################################
x_ord1 = DateScale()
y_sc1 = LinearScale()

bar = Bars(scales={'x': x_ord1, 'y': y_sc1}, colors=['blue'], padding=0.2)

ax_x = Axis(scale=x_ord1, **axes_options_1["x"])
ax_y = Axis(scale=y_sc1, orientation='vertical', tick_format='0.1e', **axes_options_1["y"])

Hourly_Power_Consumption_fig = Figure(marks=[bar], axes=[ax_x, ax_y], title="Hourly Power Consumption", 
             animation_duration=1000, legend_style=dict(fill="var(--jp-layout-color0)"),
             legend_location="bottom-left")
Hourly_Power_Consumption_fig.axes[0].tick_style = {'font-size': '10px'}  # Pour l'axe x
Hourly_Power_Consumption_fig.axes[1].tick_style = {'font-size': '10px'} 
Hourly_Power_Consumption_fig.layout.width = '50%'  # Set width to 50%
axes = plt.axes(options=axes_options_1)

################################ Import Export Chart ##############################################

x_ord = DateScale()
y_sc = LinearScale()

Exp_bar = Bars(scales={'x': x_ord, 'y': y_sc}, colors=['blue'], padding=0.2)
Imp_bar = Bars(scales={'x': x_ord, 'y': y_sc}, colors=['rgba(255, 0, 0, 0.5)'], padding=0.2)

ax_x = Axis(scale=x_ord,**axes_options_2["x"])
ax_y = Axis(scale=y_sc, orientation='vertical', tick_format='0.1f',**axes_options_2["y"])

fig = Figure(marks=[Exp_bar, Imp_bar], axes=[ax_x, ax_y], title="Import Export Chart",animation_duration=1000, 
             legend_style=dict(fill="var(--jp-layout-color0)"),legend_location="bottom-left",axes_options=axes_options_2)
axes = plt.axes(options=axes_options_2)


legend_html = '<div style="margin-top:10px;"><span style="color:blue">&#9632;</span> Export Power</div>' \
              '<div><span style="color:red">&#9632;</span> Import Power</div>'
legend_widget = HTML(value=legend_html)

fig.axes[0].tick_style = {'font-size': '10px'}  # Pour l'axe x
fig.axes[1].tick_style = {'font-size': '10px'} 
fig_with_legend = VBox([fig, legend_widget])
fig_with_legend.layout.width = '50%'  # Set width to 50%

################################ Power by source chart #############################################
x_scale = OrdinalScale()
y_scale = LinearScale()

bar_S = Bars(scales={"x": x_scale, "y": y_scale}, padding=0.2, colors=CATEGORY10, color_mode="group")

ax_x = Axis(scale=x_scale, **axes_options_3["x"])
ax_y = Axis(scale=y_scale, orientation="vertical", tick_format="0.2f", **axes_options_3["y"])

bar_S.orientation = "horizontal"
ax_x.orientation = "vertical"
ax_y.orientation = "horizontal"

source_fig = Figure(title='Power Consumption by Source', axes=[ax_x, ax_y], animation_duration=1000, 
                    axes_options=axes_options_3)
source_fig.axes[0].tick_style = {'font-size': '6px', 'font-weight': 'bold'}  # Pour l'axe x
source_fig.axes[1].tick_style = {'font-size': '10px'}  # Pour l'axe x

source_fig.marks = [bar_S]
##############################Power imported by country#############################################
x_ord3 = OrdinalScale()
y_sc3 = LinearScale()

bar_C = Bars(scales={"x": x_ord3, "y": y_sc3}, padding=0.1, colors=CATEGORY10, color_mode="group")
ax_x = Axis(scale=x_ord3,**axes_options_4["x"])
ax_y = Axis(scale=y_sc, orientation="vertical", tick_format="0.2f",**axes_options_4["y"])

'''bar.orientation = "horizontal"
ax_x.orientation = "vertical"
ax_y.orientation = "horizontal"'''
source_fig_c = Figure(title='Power Imported by Country', axes=[ax_x, ax_y], animation_duration=1000, 
                    axes_options=axes_options_3)
source_fig_c.axes[0].tick_style = {'font-size': '10px'}  # Pour l'axe x
source_fig_c.axes[1].tick_style = {'font-size': '10px'}  # Pour l'axe x

source_fig_c.marks = [bar_C]
#######################################################################################################
def handle_click(event, feature, **kwargs):
    country = feature['properties']['ADMIN']
    task= asyncio.create_task(fct.Hourly_Power_Consumption(country, bar, token))
    task1= asyncio.create_task(fct.Hourly_Imp_Exp_Power_Consumption(country, Imp_bar, Exp_bar, token))
    task2=asyncio.create_task(fct.Power_Consumption_Source(country,bar_S,token))
    task3=asyncio.create_task(fct.Power_Imported_Country(country,bar_C,token))


geo_json_layer.on_click(handle_click)

# Arrange the charts horizontally using HBox
charts_hbox = HBox([Hourly_Power_Consumption_fig,fig_with_legend])

box.children = [m, charts_hbox,source_fig,source_fig_c]  # Add charts_hbox to the VBox

#box


In [7]:
w = {"Map": m, "Electricity Production": charts_hbox, "Electricity by Origin ": source_fig,"Electricity imported by country":source_fig_c}
header = dict(title="Electricity Production", buttons=[])
FlexLayout(
    w,
    style={"height": "calc(160vh - 100px)"},
    header=header,
    template="Electricity_map.json",
)

FlexLayout(children={'Map': Map(bottom=544.0, center=[50.736455137010665, 0.3515625], controls=(ZoomControl(op…